# COVID-19 cases

### Load python tools

In [1]:
import pandas as pd
import geopandas as gpd
import jenkspy
import matplotlib.pyplot as plt
%matplotlib inline
import json
import numpy as np
from altair import datum
import altair as alt
import altair_latimes as lat
alt.themes.register('latimes', lat.theme)
alt.themes.enable('latimes')
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()
from selenium import webdriver
driver = webdriver.Chrome(executable_path='/Users/mhustiles/Desktop/chromedriver')

### CDC testing capacity

In [20]:
tables = pd.read_html('https://www.cdc.gov/coronavirus/2019-ncov/cases-updates/testing-in-us.html?CDC_AA_refVal=https%3A%2F%2Fwww.cdc.gov\
%2Fcoronavirus%2F2019-ncov%2Ftesting-in-us.html',  header=0)

In [21]:
tables = pd.DataFrame(tables[0])

In [22]:
tables.columns = tables.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [23]:
tables.rename(columns={"us_public_health_labs": "US Public Health Labs",\
                   "cdc_labs": "CDC Labs", 'date_collected':'Date'}, inplace=True)

In [24]:
tables['Date'] = pd.to_datetime(tables['Date'] + '/2020')

In [25]:
# tables = tables[tables['Date'] <= '2020/03/10']

In [26]:
tables_melt = pd.melt(tables, id_vars=['Date'], value_vars=['US Public Health Labs', 'CDC Labs'],
        var_name='lab_type', value_name='test_count')

In [27]:
tables.to_csv('output/cdc_testing_table.csv')
tables_melt.to_csv('output/cdc_testing_table_melt.csv')

In [28]:
tables_melt['test_count'] = tables_melt['test_count'].str.replace('‡','').astype(int)

In [29]:
tables_melt.head()

,Date,lab_type,test_count
0,2020-01-18,US Public Health Labs,0
1,2020-01-19,US Public Health Labs,0
2,2020-01-20,US Public Health Labs,0
3,2020-01-21,US Public Health Labs,0
4,2020-01-22,US Public Health Labs,0


In [30]:
chart = alt.Chart(tables_melt).mark_bar(size=12).encode(
    x=alt.X('Date:T', title='Date', axis=alt.Axis(tickCount=10)),
    y=alt.Y('sum(test_count)', title=''),
    color=alt.Color('lab_type', legend=alt.Legend(orient="top"), title=''),
    order=alt.Order(
      # Sort the segments of the bars by this field
      'lab_type',
      sort='ascending'
    )
).properties(width=800, title={
      "text": ["CDC: Daily U.S. coronavirus specimens tested, by lab type"], 
      "subtitle": ["Figures from last four days reported as 'pending'"],
      "color": "black",
      "subtitleColor": "gray"})
  

In [32]:
chart.save('visualization.png')

In [ ]:
chart.save('visualization.svg')

---

In [103]:
url_cases = 'https://www.cdc.gov/coronavirus/2019-ncov/map-data-cases.csv'

In [ ]:
cases = pd.read_csv(url_cases)

In [ ]:
cases.columns = cases.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [120]:
cases['cases_reported'] = cases['cases_reported'].str.replace('None', '0')

In [121]:
cases.sort_values(by='cases_reported',ascending=False).head(50)

,name,range,cases_reported,community_transmission,url
34,New Jersey,51 to 100,98,Undetermined,http://www.state.nj.us/health/
15,Illinois,51 to 100,93,Yes,http://www.idph.state.il.us/
48,South Dakota,6 to 10,9,Undetermined,http://doh.sd.gov/
41,Oklahoma,6 to 10,9,Undetermined,https://www.ok.gov/health/
18,Kansas,6 to 10,8,Undetermined,http://www.kdheks.gov/
8,Delaware,6 to 10,8,Undetermined,http://www.dhss.delaware.gov/dhss/dph/index.html
55,Washington,501 to 1000,708,Yes,http://www.doh.wa.gov/
30,Montana,6 to 10,7,Undetermined,http://dphhs.mt.gov/
13,Hawaii,6 to 10,7,Undetermined,http://health.hawaii.gov/
50,Texas,51 to 100,67,Undetermined,https://www.dshs.state.tx.us/


---

### Johns Hopkins data

In [ ]:
# Cases: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/1

# Deaths: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/0

# Cases countries: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/Coronavirus_2019_nCoV_Cases/FeatureServer/2

# Cases/time: 
# https://services1.arcgis.com/0MSEUqKaxRlEPj5g/ArcGIS/rest/services/cases_time_v3/FeatureServer/0

---

In [54]:
county_pop = pd.read_csv('input/processed/acs5_2018_population_counties.csv',\
                        dtype={'geoid':'object', 'state':'object', 'county':'object'})

In [55]:
ca_counties = pd.DataFrame(county_pop[county_pop['state'] == '06'])

In [56]:
ca_counties.drop(columns=['universe_annotation', 'universe_moe', 'universe_moe_annotation'], inplace=True)
ca_counties.rename(columns={"universe": "population"}, inplace=True)

In [57]:
ca_counties['county'] = ca_counties['name'].str.replace(' County, California', '')

In [58]:
ca_counties.head()

,geoid,name,population,state,county
181,06001,"Alameda County, California",1643700.0,06,Alameda
182,06005,"Amador County, California",37829.0,06,Amador
183,06007,"Butte County, California",227075.0,06,Butte
184,06009,"Calaveras County, California",45235.0,06,Calaveras
185,06011,"Colusa County, California",21464.0,06,Colusa


In [59]:
icus_county_pop = utilization_counties.merge(ca_counties, on='county')

In [60]:
bayarea = ['Alameda', 'Marin', 'Contra Costa', 'San Mateo',\
           'Santa Clara', 'San Francisco', 'Sonoma', 'Napa', 'Solano']

In [61]:
bayarea_counties = icus_county_pop[icus_county_pop['county'].isin(bayarea)]
bayarea_counties.population.sum()

7675798.0

In [62]:
bayarea_counties.icu_beds.sum()

1478.0

In [63]:
bayarea_counties.head(9)

,county,icu_days,icu_bed_days,icu_beds,icu_occupancy_rate,geoid,name,population,state
0,Alameda,55392.0,96360.0,264.0,58.133333,06001,"Alameda County, California",1643700.0,06
5,Contra Costa,42042.0,61902.0,169.0,67.250000,06013,"Contra Costa County, California",1133247.0,06
19,Marin,6931.0,10950.0,30.0,67.666667,06041,"Marin County, California",260295.0,06
26,Napa,6158.0,17520.0,48.0,42.000000,06055,"Napa County, California",140530.0,06
36,San Francisco,63036.0,122925.0,347.0,51.083333,06075,"San Francisco County, California",870044.0,06
39,San Mateo,17945.0,35040.0,96.0,51.666667,06081,"San Mateo County, California",765935.0,06
41,Santa Clara,84576.0,140160.0,370.0,54.545455,06085,"Santa Clara County, California",1922200.0,06
45,Solano,14928.0,29930.0,82.0,53.600000,06095,"Solano County, California",438530.0,06
46,Sonoma,17448.0,26280.0,72.0,57.000000,06097,"Sonoma County, California",501317.0,06


In [64]:
icus_county_pop.head(10)

,county,icu_days,icu_bed_days,icu_beds,icu_occupancy_rate,geoid,name,population,state
0,Alameda,55392.0,96360.0,264.0,58.133333,06001,"Alameda County, California",1643700.0,06
1,Amador,1464.0,2190.0,6.0,67.000000,06005,"Amador County, California",37829.0,06
2,Butte,8504.0,17155.0,59.0,48.666667,06007,"Butte County, California",227075.0,06
3,Calaveras,791.0,2920.0,8.0,27.000000,06009,"Calaveras County, California",45235.0,06
4,Colusa,20.0,2190.0,6.0,1.000000,06011,"Colusa County, California",21464.0,06
5,Contra Costa,42042.0,61902.0,169.0,67.250000,06013,"Contra Costa County, California",1133247.0,06
6,Del Norte,1755.0,2190.0,6.0,80.000000,06015,"Del Norte County, California",27424.0,06
7,El Dorado,3738.0,10220.0,28.0,36.000000,06017,"El Dorado County, California",186661.0,06
8,Fresno,35313.0,54385.0,149.0,65.750000,06019,"Fresno County, California",978130.0,06
9,Glenn,0.0,0.0,0.0,NaN,06021,"Glenn County, California",27897.0,06


In [65]:
icus_county_pop['icus_per1000'] = (icus_county_pop['icu_beds'] * 100000) / icus_county_pop['population']

In [66]:
icus_county_pop.sort_values(by='icus_per1000', ascending=False).head()

,county,icu_days,icu_bed_days,icu_beds,icu_occupancy_rate,geoid,name,population,state,icus_per1000
36,San Francisco,63036.0,122925.0,347.0,51.083333,06075,"San Francisco County, California",870044.0,06,39.883040
43,Shasta,11566.0,25915.0,71.0,45.500000,06089,"Shasta County, California",179085.0,06,39.645978
26,Napa,6158.0,17520.0,48.0,42.000000,06055,"Napa County, California",140530.0,06,34.156408
18,Madera,9598.0,18980.0,52.0,51.000000,06039,"Madera County, California",155013.0,06,33.545574
55,Yuba,5873.0,8760.0,24.0,67.000000,06115,"Yuba County, California",75493.0,06,31.791027


In [67]:
icus_county_pop_slim = icus_county_pop[[ 'county', 'icu_beds', 'icus_per1000']]

In [68]:
icus_county_pop_slim.to_csv('output/icus_county_pop_slim.csv')

---

In [69]:
counties_geo = gpd.read_file('/Users/mhustiles/data/data/GIS/LA/counties/2012/counties.shp')

In [70]:
counties_geo = counties_geo.to_crs({'init': 'epsg:4326'})

In [71]:
counties_geo.columns = counties_geo.columns.str.strip().str.lower().str.replace(' ', '_')\
                    .str.replace('(', '').str.replace(')', '').str.replace('-','_')

In [72]:
counties_geo.rename(columns={"fips": "geoid"}, inplace=True)

In [73]:
counties_geo_slim = counties_geo[[ 'geoid', 'name', 'geometry']]

In [74]:
icus_geo = counties_geo_slim.merge(icus_county_pop, on='geoid')

In [75]:
icus_geo.to_file('output/icus_geo.geojson', driver='GeoJSON')

In [76]:
# !tippecanoe --generate-ids --force -r1 -pk -pf -o \
# output/icus_geo.mbtiles \
# output/icus_geo.geojson